In [2]:
import pandas as pd
import numpy as np
from gurobipy import *

In [5]:
# cargar la cinética de recuperacion
data_name = 'newman1'
cinetica_path = '../Daniel_inputs/'
block_path = '../minelib_inputs/' + data_name + '.blocks'
prec_path = '../minelib_inputs/'+ data_name +'.prec'
cinetica_file = 'REC_MODEL_T240.csv'
cinetica_df = pd.read_csv(cinetica_path + cinetica_file)
blocks_df = pd.read_csv(block_path, header=None)
rec = cinetica_df['rec90'] # panda series para la recuperacion en t=90

In [6]:
# histograma de la recuperacion en el instante 90
hist = cinetica_df.hist(column='rec90', bins=5)
# calcular nuevos valores de la funcion objetivo
# usando la recuperación al dia 90
nblocks = blocks_df.shape[0]
# algunos datos dados
c_conv = 2204.62
rec_AU = 0.88
price_AU = 12
seilling_AU = 0.2
rec_CU = 0.77
price_CU = 20
seilling_CU = 7.2
proc_cost = 4
mine_cost = 0.9 # por tonn
bv_list_cinetic = list()
bv_list_calc = list()
economic_value_list = list()
economic_value_rec_list = list()
with open('marvin_cinetica.upit', 'w') as output, open(block_path, 'r') as bm:
    output.write('NAME: marvin\nTYPE: UPIT\nNBLOCKS: 53271\nOBJECTIVE_FUNCTION: \n')
    for i in range(nblocks):
        #recuperar datos de bm para calcular valor de cada bloque
        linea_bm = bm.readline()
        linea_lista = linea_bm.split()
        tonn = float(linea_lista[4])
        AU = float(linea_lista[5])
        CU = float(linea_lista[6])
        economic_value = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec[i]/100*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_rec = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec_CU*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_list.append(economic_value)
        economic_value_rec_list.append(economic_value_rec)

ValueError: could not convert string to float: 'FRWS'

In [20]:
# optimo con recuperacion del ultimo dia 
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 3052245.7535
Found heuristic solution: objective 2.155104e+07
Found heuristic solution: objective 3.447631e+07
Presolve removed 587607 rows and 46800 columns (presolve time = 5s) ...
Presolve removed 617931 rows and 49095 columns
Presolve time: 6.49s
Presolved: 32700 rows, 4176 columns, 65400 nonzeros
Found heuristic solution: objective 8.477468e+08
Variable types: 0 continuous, 4176 integer (4176 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5500000e+09   0.000000e+00   4.745000e+03      7s
    5515    1.2375244e+09   0.000000e+00   0.000000e+00      7s
    5515    1.2375244e+09   0.000000e+00   0.000000e

In [25]:
# optimo con recuperacion cte
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_rec_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 2796059.4414
Found heuristic solution: objective 2.031152e+07
Found heuristic solution: objective 3.235205e+07
Presolve removed 584413 rows and 46463 columns (presolve time = 5s) ...
Presolve removed 614385 rows and 48755 columns
Presolve time: 6.56s
Presolved: 36246 rows, 4516 columns, 72492 nonzeros
Found heuristic solution: objective 7.999334e+08
Variable types: 0 continuous, 4516 integer (4516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5643467e+09   0.000000e+00   5.157000e+03      7s
    5659    1.2406584e+09   0.000000e+00   0.000000e+00      7s
    5659    1.2406584e+09   0.000000e+00   0.000000e

In [7]:
# obtener funcion objetivo del .pcpsp, para comparar
# el resultado del modelo gurobi con el modelo AMPL
data_name = 'newman1'
pcpsp_path = '../minelib_inputs/' + data_name + '.pcpsp'
prec_path = '../minelib_inputs/' + data_name + '.prec'
upit_path = '../minelib_inputs/'+ data_name + '.upit'
objective_function_pcpsp = {}
resource_constraint_ub_limits = {}
resource_constraint_lb_limits = {}
resource_constraint_coefficients = {}
with open(pcpsp_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        if linea_lista[0] == 'NAME:':
            dato = linea_lista[1].strip('\n')
            name = dato
        elif linea_lista[0] == 'NBLOCKS:':
            dato = linea_lista[1].strip('\n')
            nblocks = int(dato)
        elif linea_lista[0] == 'NPERIODS:':
            dato = linea_lista[1].strip('\n')
            nperiods = int(dato)
        elif linea_lista[0] == 'NDESTINATIONS:':
            dato = linea_lista[1].strip('\n')
            ndestinations = int(dato)
        elif linea_lista[0] == 'NRESOURCE_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            nresource_side_constraints = int(dato)
        elif linea_lista[0] == 'NGENERAL_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            ngeneral_side_constraints = int(dato)
        elif linea_lista[0] == 'DISCOUNT_RATE:':
            dato = linea_lista[1].strip('\n')
            discount_rate = float(dato)
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_LIMITS:':
            for r in range(nresource_side_constraints):
                for t in range(nperiods):
                    linea = f.readline()
                    lista = linea.split()
                    resource_constraint_ub_limits[r,t] = int(lista[-1])
                    resource_constraint_lb_limits[r,t] = '-Infinity' # falta hacer para el caso general
    
        elif linea_lista[0] == 'OBJECTIVE_FUNCTION:':
            for b in range(nblocks):
                linea = f.readline()
                lista= linea.split()
                objective_function_pcpsp[b,0] = float(lista[1])
                objective_function_pcpsp[b,1] = float(lista[2])
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_COEFFICIENTS:':
                for linea in f:
                    if linea == 'EOF\n':
                        break
                    lista = linea.split()
                    b = int(lista[0])
                    d = int(lista[1])
                    r = int(lista[2])
                    resource_constraint_coefficients[b,r,d] = float(lista[3])

# llenar con ceros las entradas de resource_constraint_coefficients
# que no está definidas
for b,r,d in itertools.product(range(nblocks), range(nresource_side_constraints),range(ndestinations)):
    if not (b,r,d) in resource_constraint_coefficients:
        resource_constraint_coefficients[b,r,d] = 0

print('Data base name: %s' % (name))
print('NBLOCKS: %d' % nblocks)
print('NPERIODS: %d' % nperiods)
print('NDESTINATIONS: %d' % ndestinations)
print('NRESOURCE_SIDE_CONSTRAINTS: %d' % nresource_side_constraints)
print('NGENERAL_SIDE_CONSTRAINTS: %d' % ngeneral_side_constraints)
print('DISCOUNT_RATE: %.2f' % discount_rate)

Data base name: Newman1
NBLOCKS: 1060
NPERIODS: 6
NDESTINATIONS: 2
NRESOURCE_SIDE_CONSTRAINTS: 2
NGENERAL_SIDE_CONSTRAINTS: 0
DISCOUNT_RATE: 0.08


In [8]:
bv_list = list() # block value list para upit
with open(upit_path, 'r') as f:
    for i in range(4):
        f.readline()
    for line in f:
        if not line == 'EOF\n':
            lista = line.split()
            bv_list.append(float(lista[1]))

In [10]:
# beneficio con valores objetivos dado por .upit
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(LinExpr([bv_list[i]for i in range(nblocks)], [x[i] for i in range(nblocks)]), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()
# recuperar upit
blocks_id = [i for i in range(nblocks) if x[i].x==1] # recuperar upit
print('UPIT BLOCKS: %d' % len(blocks_id))

Optimize a model with 3922 rows, 1060 columns and 7844 nonzeros
Variable types: 0 continuous, 1060 integer (1060 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 989307.55494
Found heuristic solution: objective 2.608690e+07
Presolve removed 3922 rows and 1060 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 2.60869e+07 989308 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.608689902597e+07, best bound 2.608689902597e+07, gap 0.0000%
UPIT BLOCKS: 1059


In [11]:
# pcpsp model: variables y restricciones
m = Model()
# definir variables para pcpsp
x_pcpsp = {}
y = {}
blocks = blocks_id
for i in blocks:
    for t in range(nperiods):
        x_pcpsp[i,t] = m.addVar(vtype=GRB.BINARY, name = "x_pcpsp%d" % i)
for i in blocks:
    for d in range(ndestinations):
        for t in range(nperiods):
            y[i,d,t] = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS)

indices = list(itertools.product(range(nperiods), range(ndestinations),blocks))
objetivo_expr = LinExpr([(1.0/(1.0 + discount_rate))**t * objective_function_pcpsp[b,d] for t,d,b in indices],
                   [y[b,d,t] for t,d,b in indices])
m.setObjective(objetivo_expr, GRB.MAXIMIZE)
for r in range(nresource_side_constraints):
    for t in range(nperiods):
        blockTimesDest = list(itertools.product(blocks,range(ndestinations)))
        resource = LinExpr([resource_constraint_coefficients[b,r,d] for b,d in blockTimesDest], 
                           [y[b,d,t] for b,d in blockTimesDest])
        m.addConstr(resource_constraint_lb_limits[r,t] <=
                   resource <= resource_constraint_ub_limits[r,t])
# CliqueBlock
for b in blocks:
    m.addConstr(LinExpr([1 for t in range(nperiods)], [x_pcpsp[b,t] for t in range(nperiods)])<=1)
# SumDest
for b in blocks:
    for t in range(nperiods):
        m.addConstr(LinExpr([1 for d in range(ndestinations)], [y[b,d,t] for d in range(ndestinations)])==x_pcpsp[b,t])
# Precedence
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        b = int(linea_lista[0])
        if b in blocks: # solo agregar precedencia para b in UPIT
            for j in range(nvecinos):
                v = int(linea_lista[j+2])
                for t in range(nperiods):
                    m.addConstr(LinExpr([1 for s in range(t+1)], [x_pcpsp[b,s] for s in range(t+1)]) <= LinExpr( [1 for s in range(t+1)], [x_pcpsp[v,s] for s in range(t+1)]))

In [12]:
m.optimize()

Optimize a model with 30957 rows, 19062 columns and 209202 nonzeros
Variable types: 12708 continuous, 6354 integer (6354 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [5e+00, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
Found heuristic solution: objective -0.0000000
Presolve removed 3576 rows and 6204 columns
Presolve time: 2.23s
Presolved: 27381 rows, 12858 columns, 187916 nonzeros
Variable types: 6630 continuous, 6228 integer (6228 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8705    7.1938837e+07   9.813702e+04   0.000000e+00      5s
   14783    4.3209498e+07   8.173205e+04   0.000000e+00     10s
   20101    2.4747723e+07   1.308199e+05   0.000000e+00     15s
   23154    2.4491836e+07   1.009763e+05   0.000000e+00     20s
   24065    2.4418598e+07   0.000000e+00   0.000000e+00     22s

Root relaxation: objective 2.441860e+07, 24065 iterations, 19.24 seconds

    

In [13]:
# imprimir cuantos bloques remueve por periodo
for t in range(nperiods):
    blocks_id_schedule = [i for i in blocks_id if x_pcpsp[i,t].x==1]
    print(len(blocks_id_schedule))

403
363
286
0
0
0
